In [1]:
!pip install transformers==4.41.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [2]:
import transformers
print(transformers.__version__)

4.41.2


In [1]:
import os
import json
import time
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True, torch_dtype=torch.float16).to(device)
model = model.eval()
# Monkey-patch the tokenizer's _pad method to handle the padding_side argument
original_pad = tokenizer._pad
def new_pad(self, encoded_inputs, max_length=None, padding_strategy="longest", pad_to_multiple_of=None, return_attention_mask=None, **kwargs):
    return original_pad(encoded_inputs, max_length, padding_strategy, pad_to_multiple_of, return_attention_mask)
tokenizer._pad = new_pad.__get__(tokenizer, type(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer

def compute_f1_score(text1, text2):
    # Tokenize and vectorize the texts
    vectorizer = CountVectorizer().fit([text1, text2])
    X = vectorizer.transform([text1, text2])

    # Compute F1 score
    # We calculate the F1 score based on the binary presence/absence of words
    return f1_score(X[0].toarray()[0], X[1].toarray()[0])

In [9]:
def evaluate(df):
  metrics = {'_id':[], 'f1_scores':[], 'latencies':[], 'memory_usages':[], 'peak_memory_usages':[], 'responses':[]}

  for index, row in df.iterrows():
    print(f"Processing {index}")
    try:
      metrics["_id"].append(row["_id"])
      prompt = "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:".format(context=row['summary'], input=row['input'])
      torch.cuda.reset_peak_memory_stats()
      start_time = time.time()
      response, _ = model.chat(tokenizer, prompt)
      metrics["responses"].append(response)
      total_time = time.time() - start_time
      print(f"Time taken {total_time}")

      metrics["latencies"].append(total_time)

      # Measure memory
      memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB

      #Measure peak memory
      peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
      metrics["memory_usages"].append(memory)
      metrics["peak_memory_usages"].append(peak_memory)
      metrics["f1_scores"].append(compute_f1_score(response, row['answers'][0]))

    except Exception as e:
      print("Failed with error: ", e)
      metrics["_id"].append(row["_id"])
      metrics["responses"].append(None)
      metrics["latencies"].append(None)
      metrics["memory_usages"].append(None)
      metrics["peak_memory_usages"].append(None)
      metrics["f1_scores"].append(None)
  return metrics


In [ ]:
import pandas as pd
import json
import torch
import time

ratios = [0.5, 0.6, 0.7]

for i in ratios:
  df = pd.read_csv(f'summarize_{int(i * 100)}.csv')
  metrics = evaluate(df)
  with open(f'metrics_{int(i * 100)}.json', 'w') as f:
    json.dump(metrics, f)